In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from decimal import *
TWOPLACES = Decimal(10) ** -2
from fractions import Fraction
from natsort import natsorted

In [2]:
# # parameters
customer_id = '90'
formatted_attribute = 'Paper Type'
customer_name='%wbmason%'
buckets = """Office Chairs"""

strategy_version_input=762
attribute_id_input=4517

In [4]:
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [5]:
buckets = [x for x in enrich_db(f'''SELECT * FROM (SELECT * FROM strategy_buckets sb WHERE strategy_version_id = {strategy_version_input}) AS active_strat
INNER JOIN strategy_buckets_attributes sba ON active_strat.id = sba.strategy_bucket_id
WHERE sba.attribute_id = {attribute_id_input}
AND sba.status = 'OPT_IN' ''')['bucket_id'].unique()]
# buckets

In [6]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
    
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))    

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def find_first_digit(s, non=False):
    for i, x in enumerate(s):
        if x.isdigit() ^ non:
            return i
    return -1

def split_digits(s, case=False):
    non = True
    while s:
        i = find_first_digit(s, non)
        if i == 0:
            non = not non
        elif i == -1:
            yield int(s) if s.isdigit() else s if case else s.lower()
            s = ''
        else:
            x, s = s[:i], s[i:]
            yield int(x) if x.isdigit() else x if case else x.lower()

def natural_key(s, *args, **kwargs):
    return tuple(split_digits(s, *args, **kwargs))

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]

def remove_exponent(num):
    return num.to_integral() if num == num.to_integral() else num.normalize()

def round_string_float(x):
    try:
        return remove_exponent(Decimal(x).quantize(TWOPLACES))
    except:
        return x  
    
def remove_frac(numbers):
    if str(numbers).find('/') > -1:  
        nom, den = numbers.split('/')
        tmp = float(nom)/float(den)
        return tmp
    else:
        return numbers
     
    
    
# Convert a fraction that is seperated by a space "1 1/3"
def remove_fraction(numbers):
    try:
        if numbers.find('/') > -1:   
            inte, fract = numbers.split(' ')
            nom, den = fract.split('/')
            tmp = float(nom)/float(den)
            num = int(inte) + tmp
            return num
        else:
            return numbers
    except:
        return numbers
    


#Convert a fraction that is seperated by a dash "1-1/2"
def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers
# tubes_new['height_new'] = tubes_new['height_new'].apply(lambda x: remove_fraction(x))

def frac2string(s):
    i, f = s.groups(0)
    f = round_string_float(Fraction(f))
    return str(int(i) + round_string_float(float(f)))

# measure='This is a fraction: 100 1/64'
# [re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, measure)]

In [8]:
df = query_from_file(file_name='./query/uncurated.sql', params=params)
df = df[df['bucket_id'].isin(buckets)]
len(df)

2788

In [16]:
regex_pattern_height=r'(?i)(Slip|Cellulose|Crinkle|Newsprint|Roll|Sheet|paper|Wadding)|()'        
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_height, x))
print(len(df[df['matches'].astype(str)!='[]']))
df[df['matches'].astype(str)!='[]']

1874


product_id external_id  \
1      434424733  5637355142   
2      434379745  5637176607   
3      434497586  5637472285   
9      434506149  5637488495   
10     434506929  5637488470   
...          ...         ...   
2782   434490545  5637167340   
2783   434408762  5637309327   
2784   434392212  5637249088   
2786   434380094  5637179690   
2787   434492129  5637167566   

                                                                               product_name  \
1      Neenah Paper Exact Brights Paper, 8 1/2" x 11", Bright Pink, 20 lb./74 gsm., 5000/CT   
2                     Universal Printout Paper, 1-Part, 18lb, 9.5 x 11, White, 2,700/Carton   
3             Dietzgen Inkjet Coated Matte Paper, 46 lbs., 8 mil thick, 24" x 100', 1 RL/CT   
9           JAM Paper Ledger Matte Paper, Tabloid, 11" x 17", 24 lb., Red, Recycled, 100/PK   
10    JAM Paper Colored Paper, Letter, 8 1/2" x 11" Letter, 24 lb., Violet, Recycled, 50/RM   
...                                                                                     ...   
2782           Pacon Riverside Construction Paper, 76 lbs., 12 x 18, Yellow, 50 Sheets/Pack   
2783                                              Roylco Antique Paper, 8-1/2" x 11", 32/PK   
2784                                          Pacon Fadeless Art Paper Rolls, 48 x 50, Wood   
2786                      Blizzard Blinding White Copy Paper, 8 1/2 x 11, 98 Bright, 500/RM   
2787         Pacon Spectra ArtKraft Duo-Finish Paper, 48 lbs., 36" x 1000 ft, Canary Yellow   

                                                                                                                                                                                                                                                                                                                                                                                                                   long_desc  \
1                                                                                                           Exact Brights Paper, 8 1/2" x 11", Bright Pink, 20 lb./74 gsm., 5000/CT•Get the benefit of color without the expense of color printing•Great for flyers, memos, newsletters, and more•Bright colored paper for everyday use•Inkjet, laser and copier guaranteed•Acid- and lignin-free for long-lasting documents   
2                                                                                                                                                                                                                                                                                        Printout Paper, 1-Part, 18lb, 9.5 x 11, White, 2,700/Carton•High quality computer printout paper at an economical price.•Acid-free.   
3     Inkjet Coated Matte Paper, 46 lbs., 8 mil thick, 24" x 100', 1 RL/CT•Instant dry coating produces good print quality, line acuity and image clarity across multiple inkjet print technologies without sacrificing print speed or color accuracy•Plug and play printing with standard onboard �coated paper� settings on most inkjet printers•Compatible with pressure sensitive, heat assist and thermal overlaminates   
9                                                                                                                                                            Ledger Matte Paper, Tabloid, 11" x 17", 24 lb., Red, Recycled, 100/PK•Bright and colorful for a stylish presence•Thick and sturdy with a smooth finish•Free of text or design to allow for customization•Great to use for DIY projects at home, school, or work   
10                                                                                                                                                     Colored Paper, Letter, 8 1/2" x 11" Letter, 24 lb., Violet, Recycled, 50/RM•Bright and colorful for a stylish presence•Thick and sturdy with a smooth finish•Free of text or design to allow for customization•Great to use for DIY projects at home, sch

In [17]:
df[df['matches'].astype(str)=='[]']

product_id external_id  \
0      434447669  5637406404   
4      434429134  5637368651   
5      434510475  5637488702   
6      519870690  5637516755   
7      434518740  5637393889   
...          ...         ...   
2772   434451935  5637406984   
2775   434475800  5637450216   
2777   434450105  5637412629   
2778   434518681  5637501253   
2785   434431528  5637362706   

                                                                        product_name  \
0                   JAM Paper Vellum Bristol Cardstock, 8 1/2 x 14, 67lb Grey, 50/PK   
4                                Springhill Index, 110 lb., 11" x 17", Blue, 1000/CT   
5       JAM Paper Matte Cardstock, Letter, 8 1/2" x 11", 65 lb., Smooth Black, 50/RM   
6     JAM Paper Legal Cardstock, 80 lb., Glossy 2 Sided, 8 1/2" x 14", White, 500/CT   
7         Domtar Lynx� Opaque Ultra Smooth Cover, 80 lb., 8 1/2" x 11",White, 250/PK   
...                                                                              ...   
2772                          Hammermill Blue Bond Colors, 11" x 17", 24 lb., 500/RM   
2775        Domtar Cougar� Digital Smooth Text, 80 lb., 8 1/2" x 11", White, 4000/CT   
2777      Mohawk Bright White Smooth Writing, 8.5" x 11", 20 lb., 94 Bright, 5000/CT   
2778                    Domtar Opaque Vellum Text, 60 lb., 11" x 17", Ivory, 2500/CT   
2785                 Domtar Opaque Vellum Text, 60 lb., 8 1/2" x 11", Cream, 5000/CT   

                                                                                                                                                                                                                                                                                                   long_desc  \
0                                                                                                                                                                                      Vellum Bristol Cardstock, 8 1/2 x 14, 67lb Grey, 50/PK•Perfect for the home, office, or school projects•Smooth finish   
4     Index, 110 lb., 11" x 17", Blue, 1000/CT•Smooth, hard finish offers excellent printability, whether you use ink or toner•Exceptional for die-cutting, scoring and folding•Create index systems, file cards, ruled forms or counter displays•110 lb. Index meets USPS regulations for 9 pt. reply cards   
5                                                             Matte Cardstock, Letter, 8 1/2" x 11", 65 lb., Smooth Black, 50/RM•Thick and sturdy with a smooth finish•Blank canvas for easy personalization•Colorful for a stylish quality•Perfect for creating homemade cards, invites, projects, and more   
6                                                           Legal Cardstock, 80 lb., Glossy 2 Sided, 8 1/2" x 14", White, 500/CT•Thick and sturdy with a smooth finish•Blank canvas for easy personalization•Colorful for a stylish quality•Perfect for creating homemade cards, invites, projects, and more   
7                              Lynx� Opaque Ultra Smooth Cover, 80 lb., 8 1/2" x 11",White, 250/PK•SFI Certified and FSC� Certified•Consistent runnability on press•96 brightness with a brilliant true-white shade•Excellent opacity for minimal show-through•Smooth, uniform surface for better resolution   
...                                                                                                                                                                                                                                                                                                      ...   
2772                                                                                                                                Blue Bond Colors, 11" x 17", 24 lb., 500/RM•30% post-consumer waste•Bond finish•FSC certified•99.99% Jam-Free Guarantee•Contributes to satisfying credit MR1 under LEED.   
2775                                                                                      Cougar� Digital Smooth Text, 80 lb., 8 1/2

In [12]:
# regex_pattern_length=r'(?i)(\d+\.\d+.?.?.?Le?n?g?t?h?\b)|(\d+.?\d+\/\d+.?.?.?Le?n?g?t?h?\b)|((?<!\.)(?<!\/)\d+(?!\.\d).?.?.?Le?n?g?t?h?\b)|((?<!\.)(?<!\/)\d+(?!\.).?.?.?Le?n?g?t?h?\b)|()'         
# df['matches_length'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_length, x))
regex_pattern_height=r'(?i)(\d+\.\d+[\W]?[\W]?[\W]?He?i?g?h?t?.?(?!Adjustable)\b)|(\d+.?\d+\/\d+[\W]?[\W]?[\W]?He?i?g?h?t?.?(?!Adjustable)\b)|((?<!\.)(?<!up.to.)(?<![\w])(?<!\/)(?<!\/)\d+(?!\.\d)(?<!\d\\?\"?\-\d\d)[\W]?[\W]?[\W]?He?i?g?h?t?.?(?!Adjustable)(?!.?setting)(?!.?position)(?!.?adjust)(?!.?-)(?!.?100%)\b)|()'        
df_cur['matches_height'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_height, x))
# regex_pattern_width=r'(?i)(\d+\.\d+.?.?.?Wi?d?t?h?\b)|(\d+.?\d+\/\d+.?.?.?Wi?d?t?h?\b)|((?<!\.)(?<!\/)\d+(?!\.\d).?.?.?Wi?d?t?h?\b)|((?<!\.)(?<!\/)\d+(?!\.).?.?.?Wi?d?t?h?\b)|()'
# df['matches_width'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_width, x))
# regex_pattern_diameter=r'(?i)((?<!x )\d+\.\d+.?.?.?Diam?e?t?e?r?\b(?! x))|((?<!x )\d+.?\d+\/\d+.?.?.?Diam?e?t?e?r?\b(?! x))|((?<!x )(?<!\.)(?<!\/)\d+(?!\.\d).?.?.?Diam?e?t?e?r?\b(?! x))|((?<!x )(?<!\.)(?<!\/)\d+(?!\.).?.?.?Diam?e?t?e?r?\b(?! x))|()'
# df['matches_diameter'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_diameter, x))
# regex_pattern_ft=r"\d'.?.?.?[LWHD]|\d\'.?diameter|(\bround\b)|(circ)|(lamp)"
# df['matches_ft'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_ft, x))
# regex_pattern_blank=r'\d'
# df['new_matches'] = df['matches_diameter'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
regex_pattern_blank=r'(?i)(\d+.?\d*\/?\d*.?I?n?c?h?"?.?x.?\d+.?\d*\/?\d*.?I?n?c?h?"?)|(\d+.?ft)|(\d+.?foot)|(\d+.?feet)|(\d+.?inch)|(\d+.?")|(\d+")|()'
df_cur['blank_matches'] = df_cur['product_name'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
df_cur['blank_matches_long'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))

In [75]:
from natsort import natsorted
regex_pattern=r'''(?i)((?<!\-)(?<!\d)\d+(?:\.\d+|.\d+\/\d+)?[^\w\d]{0,3}He?i?g?h?t?(?!\sLabel)(?!.Adjustment)(?!.Adjustable)\b)|((?<!\.)(?<!up.to.)(?<![\w])(?<!\/)(?<!\/)(?<!\-)\d+(?!\.\d)(?<!\d\\\"\-\d\d)[\W]?[\W]?[\W]?He?i?g?h?t?(?!Adjustable)(?!\sLabel)(?!.?setting)(?!.?position)(?!.?adjust)(?!.?-)(?!.?100%)\b)|()'''           
df_cur['matches'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
print(len(df_cur[df_cur['matches'].astype(str)!='[]']))

125


In [76]:
regex_pattern_blank=r'\d'
df_cur['new_matches'] = df_cur['matches'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
height_new=df_cur[(df_cur['new_matches'].astype(str)!='[]')&(df_cur['matches'].astype(str)!='[]')]

height_new['matches']=height_new['matches'].apply(lambda x: natsorted(x))
height_new['matches']=height_new['matches'].apply(lambda x: re.sub(r'(?i)height',' in',str(x))).apply(lambda x: re.sub(r'(?i)height|\\" H|"H|H',' in',str(x))).apply(lambda x: re.sub(r'\\" H|"H|\"xa0H|xa0H|"|�|•','',str(x))).apply(lambda x: re.sub(r"\[|\]|'|\\",'',str(x))).apply(lambda x: re.sub(r"-",' ',str(x)))
height_new['matches']=height_new['matches'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)])

height_new['matches']=height_new['matches'].apply(lambda x: re.sub('\\\" H',' in',str(x))).apply(lambda x: re.sub('\\\\','',str(x))).apply(lambda x: re.sub(',','","',str(x))).apply(lambda x: re.sub('" | "','"',str(x)))
import ast
def remove_duplicates(A):
    [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
    return A
height_new['matches']=height_new['matches'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)]))
height_new['matches']=height_new['matches'].apply(lambda x: str(x)).apply(lambda x: re.sub("'",'"',str(x))).apply(lambda x: re.sub('" | "','"',str(x))).apply(lambda x: re.sub('  ',' ',str(x))).apply(lambda x: re.sub('\”|\)|inig','',str(x))).apply(lambda x: re.sub('  ',' ',str(x)))
del height_new['new_matches']

height_new['matches']=height_new['matches'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(str(x))]))
height_new['matches']=height_new['matches'].apply(lambda x: natsorted(ast.literal_eval(re.sub(r"'",'',str(re.sub(r'""','"',str(re.sub(r'" ','"',str(re.sub(r'  ',' ',str(re.sub(r',','","',str(re.sub(r'\]','"]',str(re.sub(r'\]','"]',str(re.sub(r'\[','["',str(x)))))))))))))))))))
height_new['matches']=height_new['matches'].apply(lambda x: re.sub(r"'",'',str(re.sub(r'""','"',str(re.sub(r'" ','"',str(re.sub(r'  ',' ',str(re.sub(r',','","',str(re.sub(r'\]','"]',str(re.sub(r'\]','"]',str(re.sub(r'\[','["',str(x)))))))))))))))))

In [77]:
height_new['matches']=height_new['matches'].apply(lambda x:re.sub(r'3.0 in','3 in',str(x)))
height_new['matches']=height_new['matches'].apply(lambda x: re.sub(r'"2.62 in","2.63 in"','"2.62 in"',str(x)))
height_new['matches']=height_new['matches'].apply(lambda x: re.sub(r'\.00','',str(x))).apply(lambda x: re.sub(r'\.10','.1',str(x))).apply(lambda x: re.sub(r'\.20','.2',str(x))).apply(lambda x: re.sub(r'\.30','.3',str(x))).apply(lambda x: re.sub(r'\.40','.4',str(x))).apply(lambda x: re.sub(r'\.50','.5',str(x))).apply(lambda x: re.sub(r'\.60','.6',str(x))).apply(lambda x: re.sub(r'\.70','.7',str(x))).apply(lambda x: re.sub(r'\.80','.8',str(x))).apply(lambda x: re.sub(r'\.90','.9',str(x)))  
print(len(height_new))
height_new[curation_col]=height_new['matches']
regex_pattern_rounding=r'(?i)(\d+\.\d\d\d)|(\d+\.\d0)|(\,)|([a-eghj-mo-su-z])|(\.(?!\d))|(\d\d\d\d)|()'           
height_new['rounding'] = height_new[curation_col].apply(lambda x: re_extract(regex_pattern_rounding, x))
height_new[height_new['rounding'].astype(str)!='[]']

125


buckets  bucket_id  product_id  external_id  \
348   Weatherstripping & Seals       6912   594896537   5637529961   
349   Weatherstripping & Seals       6912   594896538   5637529962   
350   Weatherstripping & Seals       6912   594896539   5637529963   
1516   Furniture Variety Packs       7491   815179415   5637561758   
3792      Wall Racks & Holders       7483   434387630   5637232049   
4236                     Lamps       7916   434437420   5637381401   

                                                                                                           product_name  \
348                                        Vestil Dock Shelter, 18" Projection, 132" x 126", For 10' x 10' Door Opening   
349                                        Vestil Dock Shelter, 24" Projection, 132" x 126", For 10' x 10' Door Opening   
350                                        Vestil Dock Shelter, 36" Projection, 132" x 126", For 10' x 10' Door Opening   
1516  Flash Furniture 23.625''W x 47.25''L Rectangular Green Plastic Height Adjustable Activity Table Set with 6 Chairs   
3792                                                                            Learning Resources Organization Station   
4236                                   Ledu Slim Line Lamp Set, Table 12 5/8" High and Floor 61 1/2" High, Silver/White   

                                                                                                                                                                                                                                                                                                                                                                                                                       long_desc  \
348                                         Dock Shelter, 18" Projection, 132" x 126", For 10' x 10' Door Opening•Three foam-filled pads offer full dock protection•Maximum compression design offers the maximum protection•Built-in air tunnels allow air to escape when contacted by a trailer•Mounting hardware and installation instructions make installation quick and easy•Fits doors sized 8' W x 8' H to 10' W x 10' H   
349                                         Dock Shelter, 24" Projection, 132" x 126", For 10' x 10' Door Opening•Three foam-filled pads offer full dock protection•Maximum compression design offers the maximum protection•Built-in air tunnels allow air to escape when contacted by a trailer•Mounting hardware and installation instructions make installation quick and easy•Fits doors sized 8' W x 8' H to 10' W x 10' H   
350                                         Dock Shelter, 36" Projection, 132" x 126", For 10' x 10' Door Opening•Three foam-filled pads offer full dock protection•Maximum compression design offers the maximum protection•Built-in air tunnels allow air to escape when contacted by a trailer•Mounting hardware and installation instructions make installation quick and easy•Fits doors sized 8' W x 8' H to 10' W x 10' H   
1516  23.625''W x 47.25''L Rectangular Green Plastic Height Adjustable Activity Table Set with 6 Chairs•Kids Table and Chair Set with 10.5" High Seats•Rectangle Table: 23.625"W x 47.25"L x 14.5-23.5"H; 220 lb. Static Load Capacity•1" Thick Smooth Laminate Top with Safety Rounded Corners•Height Adjustable Steel Legs adjust in 1" increments•Plastic Stack Chair: 12.625"W x 13.5"D x 20"H; 154 lb. Static Load Capacity   
3792                                                                                                                                                                                                                                                             Organization Station•27 Pockets that each measure 9" long by 8 1/2" high•Book sand worksheets displayed are not included•Complete Dimensions: 28 1/4" w x 45" h   
4236                                  Slim Line Lamp Set, Table 12 5/8" High and Floor 61 1/2" High, Silver/White•Simple, contemporary design blends s

In [78]:
matchesnew=height_new[['external_id',curation_col]]
matchesnew[curation_col].explode().value_counts()
matchesnew

external_id                    Q:height
0      5637347622                   ["22 in"]
1      5637363087                 ["11.5 in"]
20     5637508368                   ["19 in"]
226    5637363086                ["14.75 in"]
258    5637510568                 ["12.2 in"]
319    5637516823                   ["40 in"]
320    5637516824                   ["40 in"]
348    5637529961            ["8 in","10 in"]
349    5637529962            ["8 in","10 in"]
350    5637529963            ["8 in","10 in"]
353    5637529941                   ["10 in"]
354    5637529942                   ["10 in"]
381    5637506685                   ["20 in"]
388    5637559949                   ["66 in"]
389    5637561329                    ["5 in"]
411    5637489395                  ["0.8 in"]
446    5637530099                   ["72 in"]
447    5637530101                   ["57 in"]
472    5637300011                 ["10.1 in"]
473    5637300012                  ["3.8 in"]
474    5637305590                  ["3.9 in"]
475    5637305591                  ["4.5 in"]
477    5637363088                 ["34.5 in"]
513    5637345072                ["67.25 in"]
681    5637469680                   ["72 in"]
682    5637469681                   ["72 in"]
703    5637468122                   ["72 in"]
776    5637468123                   ["72 in"]
782    5637345069                ["47.75 in"]
1504   5637574865                   ["70 in"]
1516   5637561758  ["5 in","10.5 in","20 in"]
1540   5637388093                ["15.75 in"]
1570   5637309691                 ["5.25 in"]
1571   5637309692                 ["5.25 in"]
1581   5637326669                    ["4 in"]
1584   5637326672                    ["5 in"]
1660   5637167405                 ["6.75 in"]
1676   5637505376                  ["7.5 in"]
1730   5637378455                   ["18 in"]
1815   5637506838                  ["7.8 in"]
1852   5637500254                 ["18.3 in"]
1853   5637500255                 ["1.63 in"]
1861   5637472885                    ["6 in"]
1862   5637546414                  ["8.7 in"]
1876   5637541730                    ["3 in"]
1877   5637541731                    ["3 in"]
1888   5637553360                 ["32.2 in"]
1889   5637553361                 ["32.2 in"]
1890   5637553362                 ["18.2 in"]
1894   5637553366                 ["21.7 in"]
1897   5637553369                 ["16.1 in"]
1898   5637553370                   ["37 in"]
1913   5637553386                 ["21.9 in"]
1929   5637553404                 ["84.3 in"]
1935   5637553469                 ["21.3 in"]
1939   5637553473                 ["70.1 in"]
1940   5637553474                 ["67.5 in"]
1968   5637559262                 ["30.3 in"]
1981   5637559777                  ["9.3 in"]
1994   5637559542                 ["19.9 in"]
2159   5637381825                ["10.88 in"]
2292   5637468147                ["10.88 in"]
2330   5637492065                 ["10.4 in"]
2344   5637505374                 ["10.4 in"]
2708   5637187732                  ["5.9 in"]
2709   5637187734                  ["3.5 in"]
2710   5637187735                  ["3.5 in"]
2711   5637209372                  ["5.6 in"]
2712   5637209369                  ["5.5 in"]
2713   5637209371                 ["8.46 in"]
2715   5637245625                  ["2.5 in"]
2716   5637245629                  ["2.5 in"]
2717   5637300008                  ["7.9 in"]
2731   5637390656                ["14.38 in"]
2737   5637454800                  ["4.6 in"]
2738   5637454801                  ["4.6 in"]
2742   5637153074                  ["2.6 in"]
2743   5637153075                  ["2.6 in"]
2744   5637153087                  ["5.7 in"]
2745   5637153088                  ["2.5 in"]
2746   5637153091                  ["2.5 in"]
2751   5637153108                  ["8.3 in"]
2765   5637524373                    ["7 in"]
2766   5637542716                    ["7 in"]
2873   5637506804                 ["23.8 in"]
2979   5637396773                    ["3 in"]
2981

In [79]:
height_new[height_new['external_id'].astype(str)=='5637153108']

buckets  bucket_id  product_id  external_id  \
2751  Label Makers       9550   434483249   5637153108   

                                                             product_name  \
2751  DYMO LetraTag 100H Label Maker, 2 Lines, 3 1/10w x 2 3/5d x 8 3/10h   

                                                                                                                                                                                                                                                                                                                                                       long_desc  \
2751  LetraTag 100H Label Maker, 2 Lines, 3 1/10w x 2 3/5d x 8 3/10h•Easy to read labels can be printed in up to 18-point font.•Accurate printing is easier with large screen for text review.•An ABC keyboard makes this handheld label maker easy for kids or adults.•With 195 symbols and a date function, this DYMO label maker provides ultimate usability.   

                                                         url  \
2751  https://wbmason.com/ProductDetail.aspx?ItemID=DYM21455   

                                               image_url      value  \
2751  https://images.wbmason.com/350/L_DYM21455_EA_P.jpg  uncurated   

     Height [Nom] Height     matches_height       blank_matches  \
2751          NaN    NaN  [8 3/10h•, 100H ]  [3 1/10w x 2 3/5d]   

      blank_matches_long     matches na_matches na_measure_matches  \
2751  [3 1/10w x 2 3/5d]  ["8.3 in"]         []                 []   

        Q:height rounding  
2751  ["8.3 in"]       []

# Compliance & Janitorial Signage

In [18]:
# df_sign=df[df['buckets'].astype(str)=='Compliance & Janitorial Signage']
# regex_pattern_blank=r'(?i)(x.?\d+.?\d\/\d")|(x.?\d+.?\d+")|(x.?\d+")|(x.?\d+.?\d\/\d'')|(x.?\d+.?\d+'')|(x.?\d+'')'
# df_sign['matches'] = df_sign['blank_matches'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
# df_sign_new=df_sign[df_sign['matches'].astype(str)!='[]']


# df_sign_new['matches']=df_sign_new['matches'].apply(lambda x: natsorted(x))
# df_sign_new['matches']=df_sign_new['matches'].apply(lambda x: re.sub(r'''(?i)''|"''',' in',str(x))).apply(lambda x: re.sub(r'(?i)height|\\" H|"H|H',' in',str(x))).apply(lambda x: re.sub(r'(?i)x| |\"xa0H|xa0H|"|�|•','',str(x))).apply(lambda x: re.sub(r'in',' in',str(x))).apply(lambda x: re.sub(r"\[|\]|'|\\",'',str(x))).apply(lambda x: re.sub(r"-",' ',str(x)))
# df_sign_new['matches']=df_sign_new['matches'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)])
# # df_new['matches']=df_new['matches'].apply(lambda x: round_string_float(x))
# df_sign_new['matches']=df_sign_new['matches'].apply(lambda x: re.sub('\\\" H',' in',str(x))).apply(lambda x: re.sub('\\\\','',str(x))).apply(lambda x: re.sub(',','","',str(x))).apply(lambda x: re.sub('" | "','"',str(x)))
# df_sign_new['matches']=df_sign_new['matches'].apply(lambda x: re.sub('''in']|']'''," in']",str(x)))
# import ast
# def remove_duplicates(A):
#     [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
#     return A
# df_sign_new['matches']=df_sign_new['matches'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)]))
# df_sign_new['matches']=df_sign_new['matches'].apply(lambda x: str(x)).apply(lambda x: re.sub("'",'"',str(x))).apply(lambda x: re.sub('" | "','"',str(x))).apply(lambda x: re.sub('  ',' ',str(x))).apply(lambda x: re.sub('\”|\)|inig','',str(x))).apply(lambda x: re.sub('  ',' ',str(x)))

# df_sign_new['matches']=df_sign_new['matches'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(str(x))]))
# df_sign_new['matches']=df_sign_new['matches'].apply(lambda x: natsorted(ast.literal_eval(re.sub(r"'",'',str(re.sub(r'""','"',str(re.sub(r'" ','"',str(re.sub(r'  ',' ',str(re.sub(r',','","',str(re.sub(r'\]','"]',str(re.sub(r'\]','"]',str(re.sub(r'\[','["',str(x)))))))))))))))))))
# df_sign_new['matches']=df_sign_new['matches'].apply(lambda x: re.sub(r"'",'',str(re.sub(r'""','"',str(re.sub(r'" ','"',str(re.sub(r'  ',' ',str(re.sub(r',','","',str(re.sub(r'\]','"]',str(re.sub(r'\]','"]',str(re.sub(r'\[','["',str(x)))))))))))))))))
# df_sign_new

# Diameter

In [19]:
# df_diam=df[df['matches_diameter'].astype(str)!='[]']

# df_diam['matches']=df_diam['matches_diameter'].apply(lambda x: natsorted(x))
# df_diam['matches']=df_diam['matches'].apply(lambda x: re.sub(r'(?i)(".?diameter| diameter|".?dia|.?dia)',' in',str(x))).apply(lambda x: re.sub(r'(?i)diam|\\" H|"H|dia',' in',str(x))).apply(lambda x: re.sub(r'\\" H|"H|\"xa0H|xa0H|"|�|•','',str(x))).apply(lambda x: re.sub(r"\[|\]|'|\\",'',str(x))).apply(lambda x: re.sub(r"-",' ',str(x)))
# df_diam['matches']=df_diam['matches'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)])
# # df_new['matches']=df_new['matches'].apply(lambda x: round_string_float(x))
# df_diam['matches']=df_diam['matches'].apply(lambda x: re.sub('\\\" H',' in',str(x))).apply(lambda x: re.sub('\\\\','',str(x))).apply(lambda x: re.sub(',','","',str(x))).apply(lambda x: re.sub('" | "','"',str(x)))
# import ast
# def remove_duplicates(A):
#     [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
#     return A
# df_diam['matches']=df_diam['matches'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)]))
# df_diam['matches']=df_diam['matches'].apply(lambda x: str(x)).apply(lambda x: re.sub("'",'"',str(x))).apply(lambda x: re.sub('" | "','"',str(x))).apply(lambda x: re.sub('  ',' ',str(x))).apply(lambda x: re.sub('\”|\)|inig','',str(x))).apply(lambda x: re.sub('  ',' ',str(x)))
# # del height_new['new_matches']

# df_diam['matches']=df_diam['matches'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(str(x))]))
# df_diam['matches']=df_diam['matches'].apply(lambda x: natsorted(ast.literal_eval(re.sub(r"'",'',str(re.sub(r'""','"',str(re.sub(r'" ','"',str(re.sub(r'  ',' ',str(re.sub(r',','","',str(re.sub(r'\]','"]',str(re.sub(r'\]','"]',str(re.sub(r'\[','["',str(x)))))))))))))))))))
# df_diam['matches']=df_diam['matches'].apply(lambda x: re.sub(r"'",'',str(re.sub(r'""','"',str(re.sub(r'" ','"',str(re.sub(r'  ',' ',str(re.sub(r',','","',str(re.sub(r'\]','"]',str(re.sub(r'\]','"]',str(re.sub(r'\[','["',str(x)))))))))))))))))
# print(len(df_diam))
# # df_diam

In [20]:
# df_diam['matches']=df_diam['matches'].apply(lambda x: re.sub(r'''(?i)22.25","in''','22.25 in',str(x)))
# df_diam['matches']=df_diam['matches'].apply(lambda x: re.sub(r'''(?i)12\? in''','12 in',str(x)))
# df_diam['matches']=df_diam['matches'].apply(lambda x: re.sub(r'''(?i)8.312''','8.31',str(x)))

# df_diam[df_diam['matches'].astype(str)=='["12? in"]']

In [21]:
# df_diam[curation_col]=df_diam['matches']
# matches_dia=df_diam[['external_id',curation_col]]
# matches_dia

In [22]:
# print(len(df))
# print(len(df[(df['matches_diameter'].astype(str)=='[]')]))
# print(len(df[(df['blank_matches'].astype(str)=='[]')]))
# print(len(df[(df['blank_matches'].astype(str)=='[]')&(df['matches_diameter'].astype(str)=='[]')]))

In [23]:
# print(len(df[(df['blank_matches'].astype(str)=='[]')&(df['matches_diameter'].astype(str)=='[]')&(df['blank_matches_long'].astype(str)=='[]')]))

In [24]:
# df[(df['matches_diameter'].astype(str)=='[]')]

# NA Values

In [25]:
regex_pattern_na=r'''(?i)((?<!Cord Measures.{10})\d+\.\d+.{0,2}['"].{0,2}[LWHD]?)|((?<!Cord Measures.{10})\d+.?\d+\/\d+.{0,2}['"].{0,2}[LWHD]?)|((?<!Cord Measures.{10})(?<!\.)(?<!\/)\d+(?!\.\d).{0,2}['"].{0,2}[LWHD]?)|((?<!Cord Measures.{10})(?<!\.)(?<!\/)\d+(?!\.).{0,2}['"].{0,2}[LWHD]?)|((?<!\.)\d+(?!\.\d).{0,3}?diam)|(measure)|()'''
regex_pattern_measure=r'(?i)(measure)|()'
df_na=df_cur
df_na['na_matches'] = df_na['long_desc'].apply(lambda x: re_extract(regex_pattern_na, x))
df_na['na_measure_matches'] = df_na['long_desc'].apply(lambda x: re_extract(regex_pattern_measure, x))
print(len(df_na[(df_na['na_matches'].astype(str)=='[]')&(df_na['blank_matches'].astype(str)=='[]')]))
df_na=df_na[(df_na['matches_height'].astype(str)=='[]')&(df_na['na_matches'].astype(str)=='[]')&(df_na['na_measure_matches'].astype(str)=='[]')&(df_na['blank_matches'].astype(str)=='[]')&(df_na['blank_matches_long'].astype(str)=='[]')]

454


In [26]:
df_na[curation_col]='n/a'
matchesna=df_na[['external_id',curation_col]]
matchesna

external_id Q:height
44     5637330042      n/a
45     5637330047      n/a
46     5637330048      n/a
47     5637367178      n/a
50     5637496938      n/a
104    5637191988      n/a
105    5637191989      n/a
106    5637191990      n/a
107    5637213576      n/a
108    5637213561      n/a
109    5637213565      n/a
110    5637213566      n/a
111    5637213567      n/a
112    5637213568      n/a
113    5637217615      n/a
114    5637219440      n/a
115    5637219469      n/a
118    5637232728      n/a
120    5637246645      n/a
121    5637246700      n/a
122    5637258723      n/a
123    5637284152      n/a
124    5637283753      n/a
125    5637283961      n/a
126    5637284016      n/a
127    5637284362      n/a
128    5637284383      n/a
129    5637284307      n/a
130    5637293592      n/a
131    5637305373      n/a
132    5637305374      n/a
133    5637305375      n/a
135    5637305395      n/a
137    5637323359      n/a
138    5637321108      n/a
139    5637321109      n/a
142    5637328839      n/a
143    5637328842      n/a
144    5637329436      n/a
145    5637336893      n/a
146    5637338445      n/a
147    5637338446      n/a
148    5637351290      n/a
149    5637351293      n/a
150    5637355302      n/a
151    5637373189      n/a
152    5637373195      n/a
153    5637373190      n/a
154    5637373191      n/a
155    5637373192      n/a
156    5637373193      n/a
157    5637373194      n/a
158    5637373196      n/a
159    5637373197      n/a
162    5637387871      n/a
166    5637437254      n/a
167    5637437255      n/a
168    5637437256      n/a
170    5637460291      n/a
171    5637457972      n/a
172    5637457974      n/a
173    5637457976      n/a
174    5637457977      n/a
184    5637464551      n/a
185    5637464552      n/a
186    5637464555      n/a
187    5637464556      n/a
188    5637464557      n/a
189    5637464558      n/a
190    5637464559      n/a
191    5637464560      n/a
192    5637464561      n/a
193    5637464562      n/a
194    5637473428      n/a
196    5637506703      n/a
197    5637506705      n/a
198    5637506706      n/a
199    5637519810      n/a
200    5637533015      n/a
201    5637519811      n/a
202    5637519812      n/a
203    5637519813      n/a
204    5637519814      n/a
205    5637519815      n/a
206    5637519816      n/a
207    5637541719      n/a
208    5637541720      n/a
209    5637541721      n/a
210    5637541722      n/a
211    5637541723      n/a
212    5637541724      n/a
216    5637345738      n/a
220    5637508235      n/a
221    5637508236      n/a
222    5637508238      n/a
223    5637508342      n/a
224    5637508343      n/a
228    5637307068      n/a
241    5637426335      n/a
242    5637437922      n/a
243    5637437926      n/a
251    5637489324      n/a
252    5637491805      n/a
253    5637492064      n/a
254    5637492068      n/a
255    5637510586      n/a
256    5637510587      n/a
259    5637511235      n/a
268    5637518950      n/a
269    5637518951      n/a
271    5637518953      n/a
274    5637518956      n/a
278    5637208264      n/a
285    5637377455      n/a
287    5637383498      n/a
288    5637383237      n/a
293    5637392186      n/a
294    5637392187      n/a
295    5637392188      n/a
297    5637396805      n/a
298    5637397250      n/a
299    5637397792      n/a
300    5637397793      n/a
301    5637397794      n/a
302    5637397795      n/a
303    5637397796      n/a
307    5637497059      n/a
312    5637516948      n/a
314    5637516818      n/a
315    5637516819      n/a
317    5637516821      n/a
318    5637516822      n/a
327    5637371872      n/a
332    5637465185      n/a
334    5637511285      n/a
335    5637511284      n/a
358    5637254716      n/a
372    5637417298      n/a
383    5637530146      n/a
385    5637548077      n/a
387    5637548080      n/a
403    5637356154      n/a
404    5637356160      n/a
405    5637356163      n/a
406    5637356169      n/a
409    5637560567      n/a
444    5637530083      n/a
445    563

In [27]:
# (\d+\.?\d*\\?\W?"?'?\W?f?t?I?n?c?h?\W?w)|(\d+\W?\d*\/\d*\\?\W?"?'?\W?f?t?I?n?c?h?\W?w)

# Min Max

In [28]:
# all_split=r'''(?i)(\d+\W?\d*\/?\d*\W?I?n?c?h?"?'?f?t?\W?[lwhd]?\b\W?x\W?\d+\W?\W?\d*\/?\W?\d*\W?I?n?c?h?"?'?f?t?\W?[lwhd]?\b\W?\W?x?\W?\d*\W?\d*\/?\d*\W?I?n?c?h?"?'?f?t?\W?[lwhd]?\b)|()'''
# pat_split_THREE=r'''(?i)(\W?\W?\d+.?\d*\/?\d*.?I?n?c?h?"?'?f?t?\W?[lwdh]?.?x.?\d+.?\d*\/?\d*.?I?n?c?h?"?'?f?t?\W?[lwdh]?.?x.?\d*.?\d+\/?\d*.?I?n?c?h?"?'?f?t?\W?[lwdh]?\w?)|()'''
# pat_split_TWO=r'''(?i)(\[\"\d+\.?\d*.?\w*.?"x\"\d+\.?\d*.?\w*.?"\])|()'''

# df['matches_split']=df['product_name'].apply(lambda x: re_extract(all_split, str(x)))
# df['matches_split_frac']=df['matches_split'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, str(x))])
# df['matches_split_frac']=df['matches_split_frac'].apply(lambda x: re.sub(r'''�''','',str(x))).apply(lambda x: re.sub(r'''(\[\"\[\')|(\[\'\[\")|(\[\"\[\\\')''',"['[\'",str(x))).apply(lambda x: re.sub(r'''(\'\]\"\])|(\\\'\]\"\])|(\"\]\'\])''','''\']']''',str(x))).apply(lambda x: re.sub(r'''\"''',' in',str(x))).apply(lambda x: re.sub(r'''(\[\'\[\\\')|(\[\'\[\')''','["',str(x))).apply(lambda x: re.sub(r'''(\\\'\]\'\])|(\'\]\'\])''','"]',str(x))).apply(lambda x: re.sub(r'''(?i)x''','"x"',str(x))).apply(lambda x: re.sub(r'''\'''',' ft',str(x))).apply(lambda x: re.sub(r''' ''','',str(x))).apply(lambda x: re.sub(r'''\\ft\\ft''','ft',str(x))).apply(lambda x: re.sub(r'''\\ft''','ft',str(x))).apply(lambda x: re.sub(r'''X''','x',str(x))).apply(lambda x: re.sub(r'''in''',' in',str(x))).apply(lambda x: re.sub(r'''ft''',' ft',str(x))).apply(lambda x: re.sub(r'''\,.*''','"]',str(x)))                                                                                                                                                             

# df['matches_split_three'] = df['matches_split_frac'].apply(lambda x: re_extract(pat_split_THREE, str(x)))
# df['matches_split_two'] = df['matches_split_frac'].apply(lambda x: re_extract(pat_split_TWO, str(x)))


# regex_pattern_blank=r'(?i)(\d+\.\d*.?w\b)|()'
# df['matches'] = df['matches_split_frac'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
# split=df[df['matches_split'].astype(str)!='[]']
# print(len(split))
# print(len(split[split['matches_split_two'].astype(str)!='[]']))
# print(len(split[split['matches_split_three'].astype(str)!='[]']))

In [29]:
# pat_call_cout=r'''(?i)(\d+\.?\d*.?.?l\b)|()'''
# split['matches']=split['matches_split_frac'].apply(lambda x: re_extract(pat_call_cout, str(x)))

# print(len(split[split['matches'].astype(str)!='[]']))
# width_call_out=split[split['matches'].astype(str)!='[]']
# width_call_out['matches']=width_call_out['matches'].apply(lambda x: re.sub(r'''(?i)h''',' in',str(x))).apply(lambda x: re.sub(r'''\[''','["',str(x))).apply(lambda x: re.sub(r'''\]''','"]',str(x))).apply(lambda x: re.sub(r'''(')''','',str(x)))
# width_call_out[curation_col]=width_call_out['matches']
# matcheswidthcallout=width_call_out[['external_id',curation_col]]

In [30]:
# # split['matches_split_two'].explode().value_counts()[0:500]
# print(len(split[split['matches_split_two'].astype(str)!='[]']))
# split_two=split[(split['matches_split_two'].astype(str)!='[]')&(split['matches'].astype(str)=='[]')]
# print(len(split_two))

In [31]:
# pat_split_width_inch=r'''(?i)(ft)|()'''
# split_two['ft']=split_two['matches_split_two'].apply(lambda x: re_extract(pat_split_width_inch, str(x)))
# split_two_inch=split_two[split_two['ft'].astype(str)=='[]']
# print(len(split_two_inch))
# split_two_inch.head(2)

In [32]:
# split['matches_split_frac'].explode().value_counts()[0:500]

In [33]:
# pat_split_length=r'''(?i)(x\"\d+\.?\d*.{0,4}"\])|()'''
# pat_split_width=r'''(?i)(\[\"\d+\.?\d*.{0,4}x)|()'''
# split_two_inch['length']=split_two_inch['matches_split_two'].apply(lambda x: re_extract(pat_split_length, str(x)))
# split_two_inch['width']=split_two_inch['matches_split_two'].apply(lambda x: re_extract(pat_split_width, str(x)))

In [34]:
# split_two_inch['length']=split_two_inch['length'].apply(lambda x: re.sub(r'''\[|\]|x|\"|in|'| ''','',str(x)))
# split_two_inch['width']=split_two_inch['width'].apply(lambda x: re.sub(r'''\[|\]|x|\"|in|'| ''','',str(x)))
# # split_two_inch

In [35]:
# split_two_inch['length']=split_two_inch['length'].apply(lambda x: re.sub(r'''12.5di''','12.5',str(x))).apply(lambda x: re.sub(r'''10\.5\?''','10.5',str(x))).apply(lambda x: re.sub(r'''0\.62\-''','0.62',str(x))).apply(lambda x: re.sub(r'''18\-''','18',str(x)))
# split_two_inch['length']=split_two_inch['length'].apply(lambda x: re.sub(r'\?|\-| ','',str(x)))
# split_two_inch['width']=split_two_inch['width'].apply(lambda x: re.sub(r'\?|\-','',str(x)))

In [36]:
# pat_call_cout=r'''(?i)(\d+\.?\d*.?.?[a-z]\b)|()'''
# split_two_inch['matches']=split_two_inch['length'].apply(lambda x: re_extract(pat_call_cout, str(x)))
# split_two_inch=split_two_inch[split_two_inch['matches'].astype(str)=='[]']

In [37]:
# print(len(split_two_inch))
# split_two_inch=split_two_inch[split_two_inch['buckets'].astype(str)!='Compliance & Janitorial Signage']
# print(len(split_two_inch))
# print(split_two_inch['buckets'].explode().value_counts())

In [38]:
# split_two_inch['length']=split_two_inch['length'].astype(float)
# split_two_inch['width']=split_two_inch['width'].astype(float)

# split_two_inch['Max'] = split_two_inch[['length', 'width']].values.max(1)
# split_two_inch['Min'] = split_two_inch[['length', 'width']].values.min(1)
# split_two_inch['Max']=split_two_inch['Max'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))

In [39]:
# split_two_inch=split_two_inch[split_two_inch['length'].astype(str)!=""]
# split_two_inch=split_two_inch[split_two_inch['width'].astype(str)!=""]
# split_two_inch['length']=split_two_inch['length'].apply(lambda x:re.sub(r' in','',str(x)))#.apply(lambda x:str(f'"{x}"'))
# split_two_inch['width']=split_two_inch['width'].apply(lambda x:re.sub(r' in','',str(x))).apply(lambda x: re.sub(r'','',str(x)))
# split_two_inch['length']=split_two_inch['length'].astype(float)
# split_two_inch['width']=split_two_inch['width'].astype(float)

# split_two_inch['Max'] = split_two_inch[['length', 'width']].values.max(1)
# split_two_inch['Min'] = split_two_inch[['length', 'width']].values.min(1)
# split_two_inch['Max']=split_two_inch['Max'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))
# split_two_inch['Min']=split_two_inch['Min'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))

In [40]:
# split_two_inch[curation_col]=split_two_inch['Max']
# msplit=split_two_inch[['external_id',curation_col]]
# msplit[curation_col]=split_two_inch[curation_col].apply(lambda x: str(f"['{x} in']")).apply(lambda x: re.sub(r"'",'"',str(x)))
# # msplit[curation_col].explode().value_counts()

In [41]:
# print(len(df))
# print(len(matchesna)+len(matchesnew)+len(matchcustom))

NameError: name 'df' is not defined

In [57]:
# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][1]
#     return name
# print(get_df_name(msplit))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][1]
    return name
print(get_df_name(matchcustom))
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchesna))
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchesnew))

matchcustom
matchesna
matchesnew


In [ ]:
stop

# send to the folder for upload

In [73]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBYmatchesna - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/{customer} - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 

In [74]:
looks_good('WBMason', attribute, pand, matchcustom)

In [60]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBYmatchesna - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/{customer} - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 

looks_good('WBMason', attribute, pand, matchesna)

In [80]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][1]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBYmatchesna - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/{customer} - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
looks_good('WBMason', attribute, pand, matchesnew)